In [1]:
import queue 

## Read from file and calculate symbol occurencies

In [3]:
file_path = 'ToEncode.txt'
file = open(file_path,'r')
text = file.read()
# this is representation of how offent each symbol is met in teh text
occurencies = {}
for symbol in text:
    if not symbol in occurencies:
        occurencies[symbol] = 1
    else:
        occurencies[symbol] = occurencies[symbol] +1
file.close()
#print(occurencies)

## Build Huffman tree and form endoding and decoding dictionaries

In [4]:
p = queue.PriorityQueue()

def fill_priorityQueue(elem_by_frequency):
    symbol_by_code = {}
    for key, value in elem_by_frequency.items():
        symbol_code[key]='';
        p.put((value, key))
        
    return symbol_by_code


def build_code(symbol_code):
    while p.qsize() > 1:
        right = p.get();
        left = p.get();
        if right is None or left is None:
            break;
        left_key = left[1]
        left_vaalue = left[0]
        right_key = right[1]
        right_value = right[0]
        
        add_bits_key(left_key,'1',symbol_code)
        add_bits_key(right_key,'0', symbol_code)
        
        p.put((right_value + left_vaalue, left_key + right_key))   
        
    decoding_dict = {v:k for k,v in symbol_code.items()}     
    return symbol_code, decoding_dict
    

def add_bits_key(key, bit, symbol_code):
    if len(key) == 1:
        symbol_code[key] = bit
    else:
        for symbol in key:
            symbol_code[symbol] = bit + symbol_code[symbol]

In [5]:
#Building encoding and decoding dictionaries

while not p.empty():
    p.get()
symbol_code = {}

symbol_code = fill_priorityQueue(occurencies)
encoding_dict, decoding_dict = build_code(symbol_code)

#print(encoding_dict)
#print(decoding_dict)

## Encodind of the text and save it in bin file

In [6]:
encoded_text = ''
for symbol in text:
    encoded_text += encoding_dict[symbol]
padding_length = 8 - len(encoded_text) % 8
encoded_text += '0' * padding_length
padding_info = '{0:08b}'.format(padding_length)

padded_encoded_text = padding_info + encoded_text
#print(encoded_text)
#print(len(encoded_text) % 8)

byte_arr = bytearray()
for i in range(0, len(padded_encoded_text),8):
    byte = padded_encoded_text[i: i+8]
    byte_arr.append(int(byte, 2))
    
output_file = 'Encoded.bin'
output = open(output_file, 'wb+')
output.truncate(0)
output.write(bytes(byte_arr))
output.close()

## Decoding of the saved bin file and save to another txt

In [7]:
file_to_decode = open(output_file, 'rb')
bits_string = ''
byte = file_to_decode.read(1)

while True:#( byte != '' ):
    if len(byte) == 0:
        break;
    byte = ord(byte) # 180
    bits = bin(byte)[2:].rjust(8,'0') # 10110100
    bits_string += bits
    byte = file_to_decode.read(1)
    
#Remove padding
zeroes_to_remove = int(bits_string[0:8],2)
bits_string = bits_string[8:]
text_to_decode = bits_string[: -1 *zeroes_to_remove]

#decode teh file
initial_text = ''
key = ''
for symbol in text_to_decode:
    key += symbol;
    if key in decoding_dict:
        initial_text += decoding_dict[key]
        key = ''

# write result in new file
result_file = 'Decoded.txt'
decoded_file = open(result_file, 'w+')
decoded_file.write(initial_text)
decoded_file.close()